# Create Folders

In [ ]:
!mkdir temp

# Import

In [ ]:
import os
import torch
import datetime
from utils import *
import py_sod_metrics
import torch.nn as nn
from Model import Unet
from Dataset import ValidationDataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Global variable

In [ ]:
IMG_WIDTH = 428
IMG_HIGHT = 240
GT_FOLDER = "./temp/"

# Test

In [ ]:
def inference(model,public_dataloader):
    floder_path = GT_FOLDER
    threshold = 174
    toimg = transforms.ToPILImage()
    resize = transforms.Resize((IMG_HIGHT, IMG_WIDTH), antialias=True)

    model.eval()
    with torch.no_grad():
        # public
        i = 0
        for data in public_dataloader:
            images = data[0]
            images = images.to(get_device())
            outputs = model(images)[0]

            for out in outputs:
                cv2_img = convert_from_image_to_cv2(toimg(resize(out)))
                _, cv2_img = cv2.threshold(cv2_img, threshold, 255, cv2.THRESH_BINARY)

                if i <2160:
                    cv2.imwrite(floder_path + f"TRA_RI_{2000000 + i}.png", cv2_img)
                else:
                    cv2.imwrite(floder_path + f"TRA_RO_{2000000 + i}.png", cv2_img)
                processbar(i+1, 4320, 30, f"{i+1}/{4320} predicted.")
                i += 1


In [ ]:
def main():
    # hyper parameter
    batch_size = 70

    # dataset
    print("Loading training dataset...")
    pub_test_set = ValidationDataset(dataset_path="./Dataset/")
    
    pub_test_dataloader = DataLoader(pub_test_set, batch_size= batch_size, shuffle= False)
    print("Loading complete.")

    # model
    model = Unet(3, 1)
    model.load_state_dict({k.replace('module.',''):v for k,v in torch.load("./model/best_model.pth").items()})
    model.to(get_device())
    model = nn.DataParallel(model, device_ids=[i for i in range(torch.cuda.device_count())])
    model.eval()
    
    # inference
    print("Start inferencing...")
    start_time = datetime.datetime.now()
    inference(model, public_dataloader=pub_test_dataloader)
    end_time = datetime.datetime.now()
    print(f"Inferencing complete.\nCost {str(end_time - start_time)}.")

    
if __name__ == "__main__":
    main()

# Check Fmeasure

In [ ]:
mask_root = "./Dataset/Training_dataset/label_img/"
pred_root = GT_FOLDER

mask_name_list = sorted(os.listdir(mask_root))
FMv2 = py_sod_metrics.FmeasureV2(
    metric_handlers={
        "fm": py_sod_metrics.FmeasureHandler(with_dynamic=True, with_adaptive=False, beta=0.3)
    }
)

for i, mask_name in enumerate(mask_name_list):
    mask_path = os.path.join(mask_root, mask_name)
    pred_path = os.path.join(pred_root, mask_name)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
    FMv2.step(pred=pred, gt=mask)
    processbar(i+1, 4320, 30, f"{i+1:4d}/{4320} img measured.")

fmv2= FMv2.get_results()

results = {
    "meanfm":fmv2["fm"]["dynamic"].mean()
}

print(results)